In [25]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import openwind
import sklearn
import tensorflow
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Conv2D
from openwind.temporal.utils import export_mono
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [26]:
# Define the frequency domain for simulation
frequencies = np.arange(50,2000,1)

# Define the player
my_player = openwind.Player("TUTORIAL_REED")

# Initialize your data
X = []
y = []

# Define the range of your artificial data
data_range = np.linspace(0.1, 1.33, 20)  # Start from 0.1 to avoid ZeroDivisionError

max_length = 0  # This will hold the maximum length of our simulations


In [27]:
for i in data_range:
    # Generate the instrument geometry
    data = [[0.0, i, 6e-3, 6e-3, 'linear'],
            [i, 1.335, 6e-3, 6e-2, 'bessel', 0.7]]

    # Compute the impedance of the instrument
    result = openwind.ImpedanceComputation(frequencies, data, temperature=25)

    # Get the impedance data
    impedance_data = np.array(result.impedance)

    # Run a short simulation
    simulation = openwind.simulate(0.5, data, player=my_player, temperature=25)  # Change simulation time to 0.5 seconds

    # Get the simulation result
    simulation_data = np.array(simulation.values['bell_radiation_pressure'])

    # Update max_length if necessary
    if simulation_data.shape[0] > max_length:
        max_length = simulation_data.shape[0]

    # Prepare your data for the deep learning model
    X.append(impedance_data)  # Use the impedance data as input features
    y.append(simulation_data)  # Use the radiation pressure as the target variable


Using Theta scheme, theta = 0.25

Starting simulation! (Current time is 03:18:35 (05/05/2024))
n_steps = 44374
12% : 7s remaining
100%! Simulation ran for 5s, and stopped at 03:18:41 (05/05/2024).
Using Theta scheme, theta = 0.25

Starting simulation! (Current time is 03:18:43 (05/05/2024))
n_steps = 41319
18% : 4s remaining
90% : 0s remaining
100%! Simulation ran for 6s, and stopped at 03:18:49 (05/05/2024).
Using Theta scheme, theta = 0.25

Starting simulation! (Current time is 03:18:51 (05/05/2024))
n_steps = 43436
22% : 3s remaining
100%! Simulation ran for 4s, and stopped at 03:18:55 (05/05/2024).
Using Theta scheme, theta = 0.25

Starting simulation! (Current time is 03:18:57 (05/05/2024))
n_steps = 37950
26% : 2s remaining
100%! Simulation ran for 4s, and stopped at 03:19:01 (05/05/2024).
Using Theta scheme, theta = 0.25

Starting simulation! (Current time is 03:19:04 (05/05/2024))
n_steps = 42259
11% : 7s remaining
100%! Simulation ran for 5s, and stopped at 03:19:09 (05/05/202

In [35]:
# Convert your data to complex numbers and then to two real numbers
X_complex = np.array([np.pad(x.astype(np.complex_), (0, max_length - len(x))) for x in X])
X_real_imag = np.stack((X_complex.real, X_complex.imag), axis=-1)

# Convert your target to numpy arrays, Pad your data so all arrays have the same length
y = np.array([np.pad(y, (0, max_length - len(y))) for y in y])

# Split your data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_real_imag, y, test_size=0.2, random_state=42)

# Define your model after data has been appended to X
inputs = Input(shape=X_train.shape[1:])  # Adjusted for 1D convolution
x = Conv1D(32, 3, activation='relu', padding='same')(inputs)
x = Conv1D(32, 3, activation='relu', padding='same')(x)
outputs = Conv1D(1, 1, activation='tanh')(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile your model
model.compile(loss='mean_squared_error', optimizer='adam')

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Train your model on each set of features and target variables
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate your model
loss = model.evaluate(X_test, y_test)

print(f'Test loss: {loss}')

# Now you can use this model to make predictions
predictions = [model.predict(features.reshape(1, *features.shape)) for features in X_test]

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - loss: 10.0332 - val_loss: 7.0203
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 10.0332 - val_loss: 7.0203
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 10.0332 - val_loss: 7.0203
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - loss: 10.0332 - val_loss: 7.0203
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 10.0332 - val_loss: 7.0203
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 10.0332 - val_loss: 7.0203
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 10.0332 - val_loss: 7.0203
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 10.0332 - val_loss: 7.0203
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 10.0332 - val_loss: 7.0203
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - loss: 10.0332 - val_loss: 7.0203
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 10.0332 - val_loss: 7.0203
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 10.0332 - val_loss: 7.0203